<font color="#dd3542">Importing required libraries</font>

In [10]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn import svm,metrics
import random
import math

<font color="#dd3542">Reading data and some further processing</font>

In [11]:
df = pd.read_csv("letter-recognition.csv")
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]
m = train;
m1=m.sample(9000)
m1=m1.replace('A','1')
m1=m1.replace(['B','C','D','E','F','G','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'],'-1')
m1
X = m1.drop(['target'],axis=1).values   # independent features
y = m1['target']
y1 = y

#replacing A with +1 and everything else with -1

<font color="#dd3542">Training initial SVM</font>



In [12]:
svc = svm.SVC(kernel='linear', C=1).fit(X, y1)
f0=svc

<font color="#dd3542">Defining Loss function</font>

In [13]:
def l(smodel,zgiven):
	ans=1-int(float(smodel.predict(zgiven.drop(['target','index'],axis=1).values[0].reshape(1,-1))[0]))*int(zgiven['target'].values)
	return ans

<font color="#dd3542">Application of the algorithm</font>

In [14]:
n2=30
k_val=2
fi=svc
sam=[]
flag=0
fcurr=fi
q=1.2
k=0
ztplus1 = m1.sample()
for i in range(k_val+1):
  if (i!=0):
    tobeusedlis=[]
    for j in range(len(sam)):
      tobeusedlis.append(int(sam[j][0]))
    tempdataset=df.iloc[tobeusedlis]
    tempdataset=tempdataset.replace('A','1')
    tempdataset=tempdataset.replace(['B','C','D','E','F','G','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'],'-1')
    xtemp=tempdataset.drop(['target'],axis=1).values
    ytemp=tempdataset['target']
    fcurr=svm.SVC(kernel='linear',C=1).fit(xtemp,ytemp)

  np=0 #n+
  nm=0 #n-
  sam.clear()
  t=1
  checkflag=0
  zt = m1.sample()
  if (int(zt['target'].values)==1):
    np=np+1
  else:
    nm=nm+1
  while (np+nm<len(m1)):
    z_star=m1.sample()
    if (checkflag==0):
      zt.insert(0,'index',0)
      checkflag=1
    #zt.insert(0,'index',0)
    z_star.insert(0,'index',0)
    if (flag==1):
      zt=ztplus1
    p1 = math.exp(-l(fcurr,zt))
    p2 = math.exp(-l(fcurr,z_star))
    alpha=p2/p1
    yt =int(zt['target'].values)
    y_star =int(z_star['target'].values)

    if (k==n2):
      flag=1
      alpha2=q*alpha
      if (alpha2>=1):
        sam.append(z_star.index)
      else:
        ran_p=random.uniform(0,1)
        if (alpha2>ran_p):
          sam.append(zstar.index)
          ztplus1=z_star
          if (y_star==1):
            np=np+1
          else:
            nm=nm+1
        else:
          k=0
        continue

    


    if (alpha>=1 and yt*y_star==1):
      ind=z_star.index
      x = z_star.drop(['target','index'],axis=1).values[0].reshape(1,-1)
      x1 = zt.drop(['target','index'],axis=1).values[0].reshape(1,-1)
      P_dash =  math.exp(-y_star)*int(fcurr.predict(x)[0])/(-yt*int(fcurr.predict(x1)[0]))
      ran_p = random.uniform(0, 1)
      flag=0
      if (P_dash>ran_p):
        sam.append(z_star.index)
        if (y_star==1):
          np=np+1
        else:
          nm=nm+1
        k=k+1
      continue

    if ((yt*y_star==-1 and alpha==1)or(alpha<1)):
      ind=z_star.index
      x = z_star.drop(['target','index'],axis=1).values[0].reshape(1,-1)
      x1 = zt.drop(['target','index'],axis=1).values[0].reshape(1,-1)
      #P_dash =  math.exp(-y_star)*int(f0.predict(x)[0])/(-yt*int(f0.predict(x1)[0]))
      ran_p=random.uniform(0,1)
      flag=0
      if (alpha>ran_p):
        sam.append(z_star.index)
        if (y_star==1):
          np=np+1
        else:
          nm=nm+1
        k=k+1
      continue



<font color="#dd3542">Retrieving the sampled dataset</font>

In [15]:
finlis=[]
for i in range(len(sam)):
  finlis.append(int(sam[i][0]))
findataset=df.iloc[finlis]

<font color="#dd3542">Training the model</font>

In [16]:
X_2 = findataset.drop(['target'],axis=1).values   # independent features
y_2 = findataset['target']
sv_neww = svm.SVC(kernel='linear', C=1).fit(X_2, y_2)

<font color="#dd3542">Printing the final accuracy</font>

In [17]:
X_orig=df.drop(['target'],axis=1).values
Y_orig=df['target']
y_pred = sv_neww.predict(X_orig)
print("Accuracy:",metrics.accuracy_score(Y_orig, y_pred))

Accuracy: 0.85225
